In [6]:
import tensorflow as tf

In [7]:
from tensorflow import keras
import numpy as np

In [8]:
fashion_mnist=keras.datasets.fashion_mnist

In [9]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [10]:
train_images=train_images/255.0
test_images=test_images/255.0

In [13]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [14]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),#min value of filter you want
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),#picks like mcq
        activation='relu',
        input_shape=(28,28,1)
    ),
      #for i in range(0,hp.Int(....)) this will also work
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),# ihave added this line extra in comparison to the old model
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [15]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [16]:
tuner_search=RandomSearch(build_model,
                         objective='val_accuracy',
                         max_trials=5,directory='output',project_name="Mnist34569Fashion")

INFO:tensorflow:Reloading Tuner from output\Mnist34569Fashion\tuner0.json


In [17]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)


Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
128               |48                |conv_1_filter
5                 |5                 |conv_1_kernel
32                |48                |conv_2_filter
5                 |5                 |conv_2_kernel
128               |48                |dense_1_units
0.001             |0.01              |learning_rate

Epoch 1/3
1688/1688 [==============================] - 414s 243ms/step - loss: 0.4217 - accuracy: 0.8486 - val_loss: 0.3087 - val_accuracy: 0.8857
Epoch 2/3
1169/1688 [===================>..........] - ETA: 1:57 - loss: 0.2837 - accuracy: 0.8935

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

In [ ]:
z=model.predict(test_images)

In [ ]:
z1=np.argmax(z,axis=1)

In [ ]:
print (z1)

In [ ]:
test_labels